In [1]:
!pip install nlpaug
!pip install nltk
!pip install sacremoses
!pip install datasets
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.9 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
import pandas as pd
import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import MarianMTModel, MarianTokenizer
from datasets import Dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter, defaultdict
import math
import copy
import random
import operator
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
import time
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from transformers.modeling_outputs import TokenClassifierOutput
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import Dataset


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_epochs=1):
        super(BaseModel, self).__init__()

        self.id2label = {0: "NEGATIVE", 1: "POSITIVE"}
        self.label2id = {"NEGATIVE": 0, "POSITIVE": 1}
        self.num_labels = 2

        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=self.num_labels, id2label=self.id2label, label2id=self.label2id)
        self.loss_fn = nn.BCELoss()
        self.classifier = nn.Linear(768, self.num_labels)
        self.activation = nn.Sigmoid()
        self.dropout = nn.Dropout(0.1)
        self.trainer = None

        self.train_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=num_epochs,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            warmup_steps=500,
            weight_decay=0.01,
            logging_strategy='steps',
            logging_steps=10,
            evaluation_strategy="epoch",
            logging_dir='./logs',
        )

    def compute_metrics(self, pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, preds, average="binary"
        )
        acc = accuracy_score(labels, preds)
        return {
            "accuracy": acc,
            "f1": f1,
            "precision": precision,
            "recall": recall,
        }

    def apply_tokenizer(self, batch):
        return self.tokenizer(
            batch["text"],
            truncation=True,
            padding=True,
            max_length=100,
            add_special_tokens=True,
        )


    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)

        cls_outputs = outputs.last_hidden_state[:, 0, :]

        dropout_output = self.dropout(cls_outputs)

        outputs = self.classifier(dropout_output)

        logits = self.activation_function(outputs).view(-1, 2).float()

        loss = None

        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return TokenClassifierOutput(loss=loss, logits=logits)


    def train(self, train_df, dev_df):
        train_hf = Dataset.from_pandas(train_df)
        dev_hf = Dataset.from_pandas(dev_df)

        tokenized_train = train_hf.map(self.apply_tokenizer, batched=True)
        tokenized_dev = dev_hf.map(self.apply_tokenizer, batched=True)

        self.trainer = Trainer(
            model=self.model,
            args=self.train_args,
            tokenizer=self.tokenizer,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_dev,
            compute_metrics=self.compute_metrics
        )

        self.trainer.train()

    def evaluate_train(self, train_df):
        input_hf = Dataset.from_pandas(train_df)
        tokenized_input = input_hf.map(self.apply_tokenizer, batched=True)
        return self.trainer.evaluate(tokenized_input)

    def evaluate_dev(self):
        return self.trainer.evaluate()

In [3]:
class MultimodalModel(nn.Module):
    def __init__(self, num_categories=10, num_countries=20, num_epochs=1):
        super(MultimodalModel, self).__init__()
        self.num_labels = 2
        self.id2label = {0: "NEGATIVE", 1: "POSITIVE"}
        self.label2id = {"NEGATIVE": 0, "POSITIVE": 1}

        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        self.model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=self.num_labels, id2label=self.id2label, label2id=self.label2id)

        self.category_embedding = nn.Embedding(num_embeddings=num_categories, embedding_dim=4)
        self.country_embedding = nn.Embedding(num_embeddings=num_countries, embedding_dim=4)

        self.text_length_linear = nn.Linear(1, 4)

        self.combined_fc = nn.Linear(768 + 4 + 4 + 4, 512)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(512, self.num_labels)
        self.loss_fn = nn.CrossEntropyLoss()

        self.train_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=num_epochs,
            per_device_train_batch_size=64,
            per_device_eval_batch_size=128,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
            evaluation_strategy='epoch',
            learning_rate=5e-5
        )

    def apply_tokenizer(self, batch):
        tokenized_inputs = self.tokenizer(
            batch["text"],
            truncation=True,
            padding=True,
            max_length=100,
            add_special_tokens=True,
        )

        tokenized_inputs['category_num'] = batch['category_num']
        tokenized_inputs['country_num'] = batch['country_num']
        tokenized_inputs['text_length'] = [[length] for length in batch['text_length']]

        return tokenized_inputs


    def compute_metrics(self, pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, preds, average="binary"
        )
        acc = accuracy_score(labels, preds)
        return {
            "accuracy": acc,
            "f1": f1,
            "precision": precision,
            "recall": recall,
        }

    def forward(self, input_ids, attention_mask=None, categories=None, countries=None, text_lengths=None, labels=None):

        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        category_features = self.category_embedding(categories)
        country_features = self.country_embedding(countries)

        text_length_features = self.text_length_linear(text_lengths.view(-1, 1))
        combined_features = torch.cat((pooled_output, category_features, country_features, text_length_features), dim=1)
        combined_features = self.dropout(combined_features)

        logits = self.classifier(combined_features)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels.view(-1))

        return logits, loss

    def train(self, train_df, dev_df):
        train_hf = Dataset.from_pandas(train_df)
        dev_hf = Dataset.from_pandas(dev_df)

        tokenized_train = train_hf.map(self.apply_tokenizer, batched=True)
        tokenized_dev = dev_hf.map(self.apply_tokenizer, batched=True)

        self.trainer = Trainer(
            model=self.model,
            args=self.train_args,
            tokenizer=self.tokenizer,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_dev,
            compute_metrics=self.compute_metrics
        )

        self.trainer.train()

    def evaluate_train(self, train_df):
        input_hf = Dataset.from_pandas(train_df)
        tokenized_input = input_hf.map(self.apply_tokenizer, batched=True)
        return self.trainer.evaluate(tokenized_input)

    def evaluate_dev(self):
        return self.trainer.evaluate()

    def predict_out(self, df):
        input_hf = Dataset.from_pandas(df)
        tokenized_input = input_hf.map(self.apply_tokenizer, batched=True)
        predictions = self.trainer.predict(tokenized_input)
        return predictions.predictions

In [3]:
columns = ['id', 'identifier', 'category', 'country_code', 'text', 'multi_label']

try:
    data_df = pd.read_csv("/content/dontpatronizeme_pcl.tsv", sep='\t', header=None, names=columns, skiprows=3, index_col='id')
except pd.errors.ParserError as e:
    print("ParserError:", e)

data_df['label'] = data_df['multi_label'].apply(lambda x: 0 if x == 0 or x == 1 else 1)
data_df = data_df.dropna()
data_df["text_length"] = data_df["text"].apply(lambda x: len(x))

In [4]:
train_ids = pd.read_csv("/content/train_semeval_parids-labels.csv")
dev_ids = pd.read_csv("/content/dev_semeval_parids-labels.csv")

In [5]:
train_df = data_df.loc[data_df.index.isin(train_ids['par_id'])]
dev_df = data_df.loc[data_df.index.isin(dev_ids['par_id'])]

In [6]:
train_df["country_num"] = pd.Categorical(train_df['country_code']).codes
dev_df["country_num"] = pd.Categorical(dev_df['country_code']).codes
train_df['category_num'] = pd.Categorical(train_df['category']).codes
dev_df['category_num'] = pd.Categorical(dev_df['category']).codes

<ipython-input-6-198d7ffbd1c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["country_num"] = pd.Categorical(train_df['country_code']).codes
<ipython-input-6-198d7ffbd1c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev_df["country_num"] = pd.Categorical(dev_df['country_code']).codes
<ipython-input-6-198d7ffbd1c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [ ]:
def downsample_neg(data_df):
    pcl = data_df[data_df['label'] == 1]
    n = len(pcl)

    downsampled = pd.concat(
        [pcl, data_df[data_df['label'] == 0][: n * 2]]
    )

    return downsampled

In [ ]:
train_df_downsampled = downsample_neg(train_df)
train_df_downsampled

,identifier,category,country_code,text,multi_label,label,text_length
id,,,,,,,
33,@@8301323,disabled,pk,Arshad said that besides learning many new asp...,2,1,244
34,@@24150149,disabled,ng,Fast food employee who fed disabled man become...,3,1,66
42,@@4591694,hopeless,jm,Vanessa had feelings of hopelessness in her fi...,3,1,295
77,@@22454828,homeless,nz,"In September , Major Nottle set off on foot fr...",3,1,129
83,@@4672144,homeless,pk,The demographics of Pakistan and India are ver...,3,1,573
...,...,...,...,...,...,...,...
1775,@@9622478,refugee,nz,Last but not the least element of culpability ...,0,0,159
1776,@@7919587,refugee,ie,"Then , taking the art of counter-intuitive non...",0,0,252
1777,@@1921089,refugee,tz,Kagunga village was reported to lack necessary...,0,0,239


##Unprocessed downsampled data

In [ ]:
model_0 = MultimodalModel(num_epochs=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_0.train(train_df_downsampled, dev_df)

Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

Map:   0%|          | 0/2093 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.494200,0.376084,0.811753,0.433908,0.303823,0.758794
2,0.481300,0.216106,0.912566,0.498630,0.548193,0.457286
3,0.367500,0.410006,0.800287,0.454308,0.306878,0.874372


In [ ]:
model_0.evaluate_train(train_df_downsampled)

Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

{'eval_loss': 0.20718041062355042,
 'eval_accuracy': 0.9273719563392108,
 'eval_f1': 0.9000577700751011,
 'eval_precision': 0.8313767342582711,
 'eval_recall': 0.9811083123425692,
 'eval_runtime': 12.1433,
 'eval_samples_per_second': 196.157,
 'eval_steps_per_second': 12.27,
 'epoch': 3.0}

In [ ]:
model_0.evaluate_dev()

{'eval_loss': 0.4100058972835541,
 'eval_accuracy': 0.8002866698518872,
 'eval_f1': 0.45430809399477795,
 'eval_precision': 0.30687830687830686,
 'eval_recall': 0.8743718592964824,
 'eval_runtime': 10.6516,
 'eval_samples_per_second': 196.495,
 'eval_steps_per_second': 12.299,
 'epoch': 3.0}

In [ ]:
del model_0

## Converting ' n't' to not and removing ' 's'  - 1

In [ ]:
model_1 = MultimodalModel(num_epochs=3)

In [ ]:
def change_rem(text):
    text = re.sub('n\'t', 'not', text)
    text = re.sub('\'s', '', text)

    return text.strip()

In [ ]:
train_df_1 = train_df_downsampled.copy(deep=True)

train_df_1['text'] = train_df_1["text"].apply(
    lambda x: change_rem(x)
)

In [ ]:
train_df_1

In [ ]:
model_1.train(train_df_1, dev_df)

In [ ]:
model_1.evaluate_train(train_df_1)

In [ ]:
model_1.evaluate_dev()

In [ ]:
del model_1

## WhiteSpace - 2

In [ ]:
model_2 = MultimodalModel(num_epochs=3)

In [ ]:
def whitespace(text):

  text = re.sub('<h>', '.', text)
  text = re.sub(" +", " ", text)

  return text.strip()

In [ ]:
train_df_2 = train_df_downsampled.copy(deep=True)

train_df_2['text'] = train_df_2["text"].apply(
    lambda x: whitespace(x)
)

In [ ]:
train_df_2

In [ ]:
model_2.train(train_df_2, dev_df)

In [ ]:
model_2.evaluate_train(train_df_2)

In [ ]:
model_2.evaluate_dev()

In [ ]:
del model_2

## Punctuation - 3

In [ ]:
model_3 = MultimodalModel(num_epochs=3)

In [ ]:
def punctuation(text):

  text = re.sub(f'[{string.punctuation}]', '', text)

  return text.strip()

In [ ]:
train_df_3 = train_df_downsampled.copy(deep=True)

train_df_3['text'] = train_df_3["text"].apply(
    lambda x: punctuation(x)
)

In [ ]:
train_df_3

In [ ]:
model_3.train(train_df_3, dev_df)

In [ ]:
model_3.evaluate_train(train_df_3)

In [ ]:
model_3.evaluate_dev()

In [ ]:
del model_3

## Stop Words - 4

In [ ]:
model_4 = MultimodalModel(num_epochs=3)

In [ ]:
def stop_words(text):

  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text)
  tokens = [word for word in tokens if word.lower() not in stop_words]
  text = ' '.join(tokens)

  return text.strip()

In [ ]:
train_df_4 = train_df_downsampled.copy(deep=True)

train_df_4['text'] = train_df_4["text"].apply(
    lambda x: stop_words(x)
)

In [ ]:
train_df_4

In [ ]:
model_4.train(train_df_4, dev_df)

In [ ]:
model_4.evaluate_train(train_df_4)

In [ ]:
model_4.evaluate_dev()

In [ ]:
del model_4

## Lemmatization - 5

In [ ]:
model_5 = MultimodalModel(num_epochs=3)

In [ ]:
def lemmatize(text):

  tokens = word_tokenize(text)
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  text = ' '.join(tokens)

  return text.strip()

In [ ]:
train_df_5 = train_df_downsampled.copy(deep=True)
train_df_5['text'] = train_df_5["text"].apply(
    lambda x: lemmatize(x)
)


In [ ]:
train_df_5

In [ ]:
model_5.train(train_df_5, dev_df)

In [ ]:
model_5.evaluate_train(train_df_5)

In [ ]:
model_5.evaluate_dev()

In [ ]:
del model_5

## Stemming - 6

In [ ]:
model_6 = MultimodalModel(num_epochs=3)

In [ ]:
def stem(text):

  tokens = word_tokenize(text)
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(word) for word in tokens]
  text = ' '.join(tokens)

  return text.strip()

In [ ]:
train_df_6 = train_df_downsampled.copy(deep=True)
train_df_6['text'] = train_df_6["text"].apply(
    lambda x: stem(x)
)

In [ ]:
train_df_6

In [ ]:
model_6.train(train_df_6, dev_df)

In [ ]:
model_6.evaluate_train(train_df_6)

In [ ]:
model_6.evaluate_dev()

In [ ]:
del model_6

## Digit Removal - 7

In [ ]:
model_7 = MultimodalModel(num_epochs=3)

In [ ]:
def digits(text):

  text = re.sub(r"\d", "", text)

  return text.strip()

In [ ]:
train_df_7 = train_df_downsampled.copy(deep=True)
train_df_7['text'] = train_df_7["text"].apply(
    lambda x: digits(x)
)

In [ ]:
train_df_7

In [ ]:
model_7.train(train_df_7, dev_df)

In [ ]:
model_7.evaluate_train(train_df_7)

In [ ]:
model_7.evaluate_dev()

In [ ]:
del model_7

## 1 + 2 + 3 + 6

In [ ]:
model_8 = MultimodalModel(num_epochs=3)

In [ ]:
def combo_1(text):
  text = re.sub('n\'t', 'not', text)
  text = re.sub('\'s', '', text)
  text = re.sub('<h>', '.', text)
  text = re.sub(" +", " ", text)
  text = re.sub(f'[{string.punctuation}]', '', text)
  tokens = word_tokenize(text)
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(word) for word in tokens]

  text = ' '.join(tokens)
  return text.strip()

In [ ]:
train_df_8 = train_df_downsampled.copy(deep=True)
train_df_8['text'] = train_df_8["text"].apply(
    lambda x: combo_1(x)
)

In [ ]:
train_df_8

In [ ]:
model_8.train(train_df_8, dev_df)

In [ ]:
model_8.evaluate_train(train_df_8)

In [ ]:
model_8.evaluate_dev()

In [ ]:
del model_8

## 2+3+6


In [ ]:
model_9 = MultimodalModel(num_epochs=3)

In [ ]:
def combo_2(text):
  text = re.sub('<h>', '.', text)
  text = re.sub(" +", " ", text)
  text = re.sub(f'[{string.punctuation}]', '', text)
  tokens = word_tokenize(text)
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(word) for word in tokens]

  text = ' '.join(tokens)
  return text.strip()

In [ ]:
train_df_9 = train_df_downsampled.copy(deep=True)
train_df_9['text'] = train_df_9["text"].apply(
    lambda x: combo_2(x)
)

In [ ]:
train_df_9

In [ ]:
model_9.train(train_df_9, dev_df)

In [ ]:
model_9.evaluate_train(train_df_9)

In [ ]:
model_9.evaluate_dev()

In [ ]:
del model_9

# 1 + 2 + 3 + 4 + 5 + 6 + 7

In [ ]:
model_10 = MultimodalModel(num_epochs=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def custom_combo(text, one=True, two=True, three=True, four=True, five=True, six=True, seven=True):

  # 1
  if (one):
    text = re.sub('n\'t', 'not', text)
    text = re.sub('\'s', '', text)

  # 2
  if (two):
    text = re.sub('<h>', '.', text)
    text = re.sub(" +", " ", text)

  # 3
  if (three):
    text = re.sub(f'[{string.punctuation}]', '', text)

  # 4
  if (four):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    text = ' '.join(tokens)

  # 5
  if (five):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    text = ' '.join(tokens)

  # 6
  if (six):
    tokens = word_tokenize(text)
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    text = ' '.join(tokens)

  # 7
  if (seven):
    text = re.sub(r"\d", "", text)

  return text.strip()



In [ ]:
train_df_10 = train_df_downsampled.copy(deep=True)
train_df_10['text'] = train_df_10["text"].apply(
    lambda x: combo_10(x)
)

In [ ]:
train_df_10

,identifier,category,country_code,text,multi_label,label,text_length
id,,,,,,,
33,@@8301323,disabled,pk,arshad said besid learn mani new aspect sport ...,2,1,244
34,@@24150149,disabled,ng,fast food employe fed disabl man becom interne...,3,1,66
42,@@4591694,hopeless,jm,vanessa feel hopeless final day increas cri he...,3,1,295
77,@@22454828,homeless,nz,septemb major nottl set foot melbourn canberra...,3,1,129
83,@@4672144,homeless,pk,demograph pakistan india similar poverti wides...,3,1,573
...,...,...,...,...,...,...,...
1775,@@9622478,refugee,nz,last least element culpabl relat isol professi...,0,0,159
1776,@@7919587,refugee,ie,take art counterintuit nonsens suprem height p...,0,0,252
1777,@@1921089,refugee,tz,kagunga villag report lack necessari social se...,0,0,239


In [ ]:
model_10.train(train_df_10, dev_df)

Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

Map:   0%|          | 0/2093 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.556300,0.462352,0.905877,0.066351,0.583333,0.035176
2,0.590200,0.286664,0.911132,0.191304,0.709677,0.110553
3,0.487400,0.329943,0.874343,0.427015,0.376923,0.492462


In [ ]:
model_10.evaluate_train(train_df_10)

Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

{'eval_loss': 0.26604190468788147,
 'eval_accuracy': 0.9118387909319899,
 'eval_f1': 0.8689138576779026,
 'eval_precision': 0.8613861386138614,
 'eval_recall': 0.8765743073047859,
 'eval_runtime': 12.926,
 'eval_samples_per_second': 184.28,
 'eval_steps_per_second': 11.527,
 'epoch': 3.0}

In [ ]:
model_10.evaluate_dev()

{'eval_loss': 0.32994264364242554,
 'eval_accuracy': 0.8743430482560918,
 'eval_f1': 0.42701525054466233,
 'eval_precision': 0.3769230769230769,
 'eval_recall': 0.49246231155778897,
 'eval_runtime': 11.8306,
 'eval_samples_per_second': 176.915,
 'eval_steps_per_second': 11.073,
 'epoch': 3.0}

In [ ]:
del model_10

In [ ]:
custom_train_dfs = []

combos = [
    [True, True, True, True, True, False, False],
    [False, True, False, True, False, True, True],
    [True, True, True, True, True, True, True],
    [False, True, True, True, True, True, False],
    [True, True, False, True, True, False, False],
    [True, True, False, True, True, False, False],
    [True, False, True, False, False, True, True],
    [False, True, False, True, True, False, True],
    [True, True, False, False, True, True, False],
    [False, False, False, False, False, False, False],
    [True, True, True, True, True, False, False],
    [False, True, False, False, False, True, True],
    [True, False, False, False, False, False, True],
    [False, False, True, False, True, False, False],
    [False, True, False, True, False, False, False],
    [True, False, False, False, True, False, False],
    [False, False, False, False, False, True, False],
    [True, False, False, False, False, False, True],
    [True, True, False, True, True, False, True],
    [False, False, False, False, False, False, False],
]

for i in range(len(combos)):
  print(f"combo {i}")
  custom_model = MultimodalModel(num_epochs=5)
  custom_train_df = train_df_downsampled.copy(deep=True)
  custom_train_df['text'] = custom_train_df["text"].apply(
       lambda x: custom_combo(x, one=combos[i][0], two=combos[i][1], three=combos[i][2], four=combos[i][3], five=combos[i][4], six=combos[i][5], seven=combos[i][6])
  )
  print(custom_model.train(custom_train_df, dev_df))
  print(custom_model.evaluate_train(custom_train_df))
  print(custom_model.evaluate_dev())
  del custom_model

combo 0


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

Map:   0%|          | 0/2093 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.567500,0.338098,0.863832,0.379085,0.334615,0.437186
2,0.521600,0.269662,0.912088,0.313433,0.608696,0.211055
3,0.501700,0.381303,0.867176,0.444000,0.368771,0.557789
4,0.230400,0.433488,0.839465,0.484663,0.348786,0.793970
5,0.136000,0.413284,0.859054,0.504202,0.378788,0.753769


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.544800,0.409081,0.832776,0.410774,0.308861,0.613065
2,0.463500,0.238353,0.911610,0.206009,0.705882,0.120603
3,0.341300,0.297299,0.887721,0.521385,0.438356,0.643216
4,0.229100,0.476741,0.843765,0.499234,0.359031,0.819095
5,0.163200,0.507395,0.876732,0.532609,0.416431,0.738693


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


None


Map:   0%|          | 0/2382 [00:00<?, ? examples/s]

{'eval_loss': 0.054824791848659515, 'eval_accuracy': 0.9869857262804366, 'eval_f1': 0.9805642633228839, 'eval_precision': 0.9762796504369539, 'eval_recall': 0.9848866498740554, 'eval_runtime': 4.3994, 'eval_samples_per_second': 541.437, 'eval_steps_per_second': 33.868, 'epoch': 5.0}
{'eval_loss': 0.5073947310447693, 'eval_accuracy': 0.8767319636884854, 'eval_f1': 0.532608695652174, 'eval_precision': 0.4164305949008499, 'eval_recall': 0.7386934673366834, 'eval_runtime': 4.1383, 'eval_samples_per_second': 505.762, 'eval_steps_per_second': 31.655, 'epoch': 5.0}


In [ ]:
train_df_aug = pd.read_csv("/content/syndelinsaug.csv")

In [ ]:
train_df_aug

,id,identifier,category,country_code,text,multi_label,label,text_length
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0,0,620
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0,0,237
2,3,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0,0,158
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0,0,162
4,5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0,0,273
...,...,...,...,...,...,...,...,...
11546,10424,@@4665292,women,jm,""" they in't in fear abortion, and i just think...",3,1,133
11547,10445,@@3923193,refugee,gb,and more one hundred fifty volunteers having s...,3,1,112
11548,10454,@@22338535,vulnerable,ie,""" We are challenged, I, turn this time of cele...",4,1,240
11549,10467,@@20282330,in-need,ng,""" She huge platform, and information go out to...",3,1,282


In [ ]:
custom_train_dfs = []

combos = [
    [True, True, False, True, True, False, False],
    [True, True, False, True, True, True, False],
    [True, False, False, False, True, False, False],
    [False, True, False, True, False, False, False],
    [False, True, False, False, False, True, True],

]

for i in range(len(combos)):
  print(f"combo {i}")
  custom_model = MultimodalModel(num_epochs=5)
  custom_train_df = train_df_aug.copy(deep=True)
  custom_train_df['text'] = custom_train_df["text"].apply(
       lambda x: custom_combo(x, one=combos[i][0], two=combos[i][1], three=combos[i][2], four=combos[i][3], five=combos[i][4], six=combos[i][5], seven=combos[i][6])
  )
  custom_model.train(custom_train_df, dev_df)
  print(custom_model.evaluate_train(custom_train_df))
  print(custom_model.evaluate_dev())
  del custom_model

combo 0


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/11551 [00:00<?, ? examples/s]

Map:   0%|          | 0/2093 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.237200,0.275847,0.902054,0.189723,0.444444,0.120603
2,0.306900,0.260159,0.913043,0.260163,0.680851,0.160804
3,0.202300,0.342731,0.914955,0.310078,0.677966,0.201005
4,0.067300,0.530597,0.913043,0.187500,0.840000,0.105528
5,0.001100,0.606096,0.913043,0.208696,0.774194,0.120603


...

Map:   0%|          | 0/2093 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.279200,0.457945,0.907788,0.067633,0.875000,0.035176
2,0.262300,0.311156,0.911610,0.192140,0.733333,0.110553
3,0.081300,0.436915,0.910655,0.217573,0.650000,0.130653
4,0.034800,0.627130,0.908266,0.111111,0.705882,0.060302
5,0.000400,0.663770,0.912088,0.185841,0.777778,0.105528


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3500 already exists and is non-empty. Saving will proceed but saved res

Map:   0%|          | 0/11551 [00:00<?, ? examples/s]

{'eval_loss': 0.008560768328607082, 'eval_accuracy': 0.9986148385421175, 'eval_f1': 0.9979818365287587, 'eval_precision': 0.9994946942900454, 'eval_recall': 0.9964735516372796, 'eval_runtime': 17.5665, 'eval_samples_per_second': 657.557, 'eval_steps_per_second': 41.101, 'epoch': 5.0}
{'eval_loss': 0.6637703776359558, 'eval_accuracy': 0.9120879120879121, 'eval_f1': 0.18584070796460178, 'eval_precision': 0.7777777777777778, 'eval_recall': 0.10552763819095477, 'eval_runtime': 3.3892, 'eval_samples_per_second': 617.546, 'eval_steps_per_second': 38.652, 'epoch': 5.0}
